## Functions that are used to read and process tristan-mp data

In [1]:
import sys
import h5py
import numpy as np
import random
from scipy.optimize import root
from scipy.special import kn # Modified Bessel function
from scipy.stats import linregress
from scipy import integrate

In [2]:
'''
This function read simulation set-up parameters.
e.g. c = readparam('c')
'''
def readparam(path,param,ind=1):
    if param=='b0':
        with h5py.File(path+'flds.tot.'+str(1).zfill(3),'r') as hf:  
            bx0   = hf['bx'][0,-1,-10]
            by0   = hf['by'][0,-1,-10]
            bz0   = hf['bz'][0,-1,-10]
            return np.sqrt(bx0**2+by0**2+bz0**2)
    else:
        with h5py.File(path+'param.'+ str(ind).zfill(3),'r') as hf:       
            return np.copy(hf[param][0])

In [3]:
'''
This function read B & E fields.
e.g. bz = readparam('bz')
'''
def readflds(path,param,ind=1):
    with h5py.File(path+'flds.tot.'+str("%03d" %(ind)),'r') as hf: 
        return np.copy(hf[param][0,:,:])

In [4]:
'''
This function read particle 4-velocity and position.
e.g. ue = readparam('ue')
'''
def readprtl(path,param,ind=1):
    with h5py.File(path+'prtl.tot.'+str("%03d" %(ind)),'r') as hf:
        return np.copy(hf[param][:])   

In [5]:
'''
Read mometum and energy spectrum data
'''
def readspect(path,param,ind=1,frame=''):
    with h5py.File(path+'spect.'+str("%03d" %(ind)),'r') as hf:
        return np.copy(hf[param+frame])

In [6]:
'''
velocity boost in x-direction to a new frame
'''
def betaBoost(vx, beta, v0, ind='x'):
    gamma = 1./np.sqrt(1.-beta**2)
    tmp   = 1-vx*beta
    if ind =='x':
        return  (vx-beta)/tmp
    else:
        return v0/gamma0/tmp

In [7]:
'''
Convert Mach number in downstream to upstream
'''
def convt(M,*data):
    Mach, ad = data
    r = (ad+1.)/(ad-1.+2./M**2)
    return M-Mach*(1.+1./(r-1.))

In [8]:
'''
Calculate simulation parameters.
'''
def cal_param(path,param):
    my_dict = {}
    me = readparam(path,'me')
    mi = readparam(path,'mi')
    ad = 5./3. # adiabatic index
    
    # Inflow velocity: beta:=v/c
    if np.abs(readparam(path,'gamma0'))>=1.:
        Vinf = np.sqrt(1.-(1./readparam(path,'gamma0')**2))
    else:
        Vinf = readparam(path,'gamma0')
    my_dict['Vinf'] = Vinf

    # Temperature : delgam= kT_i/m_i c^2. Thermal velocity in unit c.
    Vthi = np.sqrt(readparam(path,'delgam'))
    Vthe = np.sqrt(readparam(path,'delgam')*readparam(path,'mi')/readparam(path,'me'))
    my_dict['Vthi'] = Vthi
    my_dict['Vthe'] = Vthe
    
    # Simulation-frame Mach number
    Mach = Vinf/np.sqrt(2.*ad*readparam(path,'delgam'))
    my_dict['Mach'] = Mach
    
    # Upstream Mach number
    sol = root(convt,Mach,args=(ad,Mach),method='hybr',tol=1e-4)
    my_dict['MachUp'] = sol.x

    # Initial B field (depend on the definition of sigma)
    Binit  = np.sqrt(readparam(path,'gamma0')*readparam(path,'ppc0')*.5*\
             readparam(path,'c')**2*me*(1+me/mi)*readparam(path,'sigma'))
    my_dict['Binit'] = Binit
    
    # Alfven speed, in unit of c
    Va = np.sqrt(readparam(path,'gamma0')*readparam(path,'sigma')*me/mi)
    my_dict['Va'] = Va
    
    # Magnetic Mach number
    my_dict['Ma'] = Vinf/Va

    # Plasma beta
    pbeta = 4.*readparam(path,'delgam')/(me/mi*(1.+me/mi)*readparam(path,'gamma0')*readparam(path,'sigma'))
    my_dict['Pbeta'] = pbeta

    # Initial Larmor radius (without gamma factor)
    Re = np.sqrt(mi/(mi+me)/readparam(path,'sigma'))/readparam(path,'c_omp')
    Ri = Re*mi/me
    my_dict['Re'] = Re
    my_dict['Ri'] = Ri
 
    return my_dict[param]

In [9]:
'''
Find shock location and shock speed 
'''
def shfind(path,ind):
    # load the final time step to find the shock's location at the end.
    dens_arr = readflds(path,'dens',ind)
    final_time = readparam(path,'time',ind)

    c_omp = readparam(path,'c_omp')
    istep = readparam(path,'istep')
    # Find out where the shock is at the last time step.
    jstart = int(min(10*c_omp/istep,dens_arr.shape[1]))
    xaxis_final = np.arange(dens_arr.shape[1])/c_omp*istep
    # Find the shock by seeing where the density is 1/2 of it's max value.

    dens1d = np.sum(dens_arr, axis=0)
    dens_half_max = max(dens_arr[dens_arr.shape[0]/2,jstart:])*.5
    dens_half_max = max(dens1d[jstart:])*.5
    # Find the farthest location where the average density is greater than half max

    #ishock_final = np.where(dens_arr[dens_arr.shape[0]/2,jstart:]>=dens_half_max)[0][-1]
    ishock_final = np.where(dens1d[jstart:]>=dens_half_max)[0][-1]
    xshock_final = xaxis_final[ishock_final]
    shock_speed = xshock_final/final_time
    prev_shock_loc = np.NaN
    return (ishock_final,xshock_final,shock_speed)

In [10]:
'''
Generating 1D spectrum (in x-direction)
'''
def spect1d(path,ind,xL,xR,shock=1,frame=''):
    # xsl particle location x-axis
    xsl   = readspect(path,'xsl',ind)/readparam(path,'c_omp')
    spece = readspect(path,'spece',ind,frame)
    specp = readspect(path,'specp',ind,frame)

    # Load gamma-1 spectrum
    gamma = readspect(path,'gamma',ind)
    # mometum := gamma*beta
    momentum = np.sqrt((gamma+1.)**2-1.)
    
    '''
    In output.F90, spece/specp is defined by the number of electrons
    and ions divided by gamma in each logrithmic energy bin, so we 
    should multiply by gamma
    '''
    for i in range(len(xsl)):
        spece[:,i] *= gamma  
        specp[:,i] *= gamma  

    # normalized energy spectra ( f = (dN/dE)/N )
    dgamma = np.empty(len(gamma))
    delta  = np.log10(gamma[-1]/gamma[0])/len(gamma) 
    for i in range(len(dgamma)):
        dgamma[i] = gamma[i]*(10.**delta-1.)
    # select x-range from which to take the spectrum reletive to shock front
    if xL >= xR:
        xR = xL + 1.
        print ('Error in spectrum range: xL<xR!')
    # ishf=index, xshf=position
    ishf,xshf,sh_speed = shfind(path,ind)
    # case=0 xL-xR regions are relative to shock.
    if shock==1:
        eL = np.searchsorted(xsl,xL+xshf)
        eR = np.searchsorted(xsl,xR+xshf,side='right')
    else:
        eL = np.searchsorted(xsl,xL)
        eR = np.searchsorted(xsl,xR,side='right')

    # energy distribution, f(E) = (dn/dE)/N
    fe = np.empty(len(gamma))
    fp = np.empty(len(gamma))
 
    norme = np.ones(len(xsl))
    normp = np.ones(len(xsl))

    # total species in each linear x bin
    for i in range(len(norme)):
        norme[i] = sum(spece[:,i])
        normp[i] = sum(specp[:,i])

    for k in range(len(fe)):
        fe[k] = sum(spece[k][eL:eR])
        fp[k] = sum(specp[k][eL:eR])

    if sum(norme[eL:eR])>0:
        fe *= 1.0/sum(norme[eL:eR])
        edist = np.copy(fe)
        fe   *= dgamma**(-1)
        femom = fe/(4.*np.pi*momentum)/(gamma+1.)
        momedist = femom*momentum**4
    else:
        print ('RUNTIME WARNING: spectrum.py cannot find electron in the integration region!')
        sys.exit()
    if sum(normp[eL:eR])>0:
        fp *= 1.0/sum(normp[eL:eR])
        pdist = np.copy(fp)
        fp   *= dgamma**(-1)
        fpmom = fp/(4.*np.pi*momentum)/(gamma+1.0)
        mompdist = fpmom*momentum**4
    else:
        print ('RUNTIME WARNING: spectrum.py cannot find ions in the integration region!')
        sys.exit()
  
    return (momentum,gamma,momedist,mompdist,edist,pdist)


In [11]:
'''
power law fit of spectrum

'''
def power_law_fit(path,ind,sp,xL,xR,GammaMin,GammaMax):
    momentum,gamma,momedist,mompdist,edist,pdist = spect1d(path,ind,xL,xR,shock=1,frame='')
    if sp=='i':
        momdist = mompdist
    elif sp=='e':
        momdist = momedist
        
    momleft  = np.sqrt((GammaMin+1)**2-1)
    momright = np.sqrt((GammaMax+1.)**2-1.)
    imLeft   = momentum.searchsorted(momleft)
    imRight  = momentum.searchsorted(momright, side='Right')

    # a while loop to make sure that the program won't mess up if mompdist == 0
    while np.abs(momdist[imLeft]) <= 1E-10 and imLeft < len(momdist)-1:
        imLeft += 1
    while np.abs(momdist[imRight]) <= 1E-10 and imRight > 0:
        imRight -=1
    if imRight>imLeft:
        pslope, pintercept, pr_value, pp_value, pstderr = \
        linregress(np.log(momentum[imLeft:imRight]), np.log(momdist[imLeft:imRight]))
        if np.isnan(pslope) == False and np.isnan(pintercept) == False:
            print ('slope: ',pslope)
            return (momentum, np.exp(pintercept)*momentum**pslope)
        else:
            print ('power law fit fails: slope/intercept is NAN!')
    else:
        print ('power law fit fails: imRight<imLeft')     
        

In [12]:
'''
Calculate the degree of polarization in x-direction.
Chi>0: right-hand polarized, Chi<0: left-hand polarized.
'''
def polarization(path,ind, xL=0, xR=1000., shock=1):
    by = readflds(path,'by',ind)
    bz = readflds(path,'bz',ind)

    xaxis_values = np.arange(by.shape[1])/readparam(path,'c_omp')*readparam(path,'istep')
    # ishf=index, xshf=position
    if shock==1:
        ishf,xshf,sh_speed = shfind(path,ind)
        xL += xshf
        xR += xshf
        # First find the left & right indices
    iL = xaxis_values.searchsorted(xL)
    iR = xaxis_values.searchsorted(xR)

    oneDslice = by.shape[0]/2

    BzFFT = np.fft.fft(bz[oneDslice,iL:iR]*readparam(path,'b0')**(-1.0))

    # Shift the fft so it is centered
    BzFFT = np.fft.fftshift(BzFFT)
    ByFFT = np.fft.fft(by[oneDslice,iL:iR]*readparam(path,'b0')**(-1.0))
    ByFFT = np.fft.fftshift(ByFFT)

    # Calculate Stokes I
    I  = ByFFT*np.conjugate(ByFFT)
    I += BzFFT*np.conjugate(BzFFT)
    # Calculate Stokes V
    V  = ByFFT*np.conjugate(BzFFT)
    V += -BzFFT*np.conjugate(ByFFT)
    V *= -1j
    # Calculate StokesChi
    StokesChi = 0.5*np.rad2deg(np.arcsin(V.real/I.real))

    # Calculate K_axis
    k_axis = np.arange(iR-iL)*(2*np.pi/(xaxis_values[1]-xaxis_values[0]))/(iR-iL-1) \
           -(2*np.pi/(xaxis_values[1]- xaxis_values[0]))*.5 
    return k_axis, StokesChi

In [13]:
'''
FFT transform of magnetic fields.
'''
def fft_flds(path,ind, param, xL=0, xR=1000, shock=1):
    pval = readflds(param,ind)
    xaxis_values = np.arange(pval.shape[1])/readparam(path,'c_omp')*readparam(path,'istep')
    # ishf=index, xshf=position
    if shock==1:
        ishf,xshf,sh_speed = shfind(ind)
        xL += xshf
        xR += xshf

    # First find the left & right indices
    iL = xaxis_values.searchsorted(xL)
    iR = xaxis_values.searchsorted(xR)
 
    oneDslice = pval.shape[0]/2
    BFFT = np.fft.fft(pval[oneDslice,iL:iR]*readparam(path,'b0')**(-1.0))
    # Shift the fft so it is centered
    BFFT = np.fft.fftshift(BFFT)
    # Calculate K_axis
    k_axis = np.arange(iR-iL)*(2*np.pi/(xaxis_values[1]-xaxis_values[0]))/(iR-iL-1) \
           -(2*np.pi/(xaxis_values[1]- xaxis_values[0]))*.5 
    return k_axis, np.abs(BFFT)

In [14]:
'''
Temperature of thermal distribution. type=0, energy, type=1, momentum
mass is in unit of me, DelGam is in unit of me*c^2
'''
def SetT(path, mass, DelGam, ind, spect_type='energy',rangex=-3,rangey=3,num=200):
    # Load gamma-1 spectrum 
    if ind<1:
        gamma = np.logspace(rangex,rangey,num)
    else:
        gamma = readspect(path,'gamma',ind)
    # Momentum := gamma*beta
    momentum=np.sqrt((gamma+1.)**2-1.)
    
    delgam0=DelGam*mass
    if delgam0 >= 0.013:
        aconst = 1/(delgam0*np.exp(1.0/delgam0)*kn(2, 1.0/delgam0))
    else:
        aconst = np.sqrt(2/np.pi)/delgam0**1.5
        aconst -= 15.0/(4.*np.sqrt(delgam0)*np.sqrt(2*np.pi))
        aconst += (345*np.sqrt(delgam0))/(64.*np.sqrt(2*np.pi))
        aconst -= (3285*delgam0**1.5)/(512.*np.sqrt(2*np.pi))
        aconst += (95355*delgam0**2.5)/(16384.*np.sqrt(2*np.pi))
        aconst -= (232065*delgam0**3.5)/(131072.*np.sqrt(2*np.pi))
        
    # energy spectral
    if spect_type=='energy':  
        fmax  = aconst*gamma*(gamma+1.0)*np.sqrt((gamma+1.0)**2-1)
        fmax *= np.exp(-gamma/delgam0)
        return gamma, fmax
    # mometum spectral
    else:
        #fmommax  = momentum**4*aconst*(gamma+1.0)*np.sqrt((gamma+1.0)**2-1)
        fmommax  = aconst*(gamma+1.0)*np.sqrt((gamma+1.0)**2-1)
        fmommax *= np.exp(-gamma/delgam0)/(4*np.pi*momentum)/(gamma+1.0)
        return momentum, fmommax


In [15]:
'''
Calculate electron and ion energy, in units me*c^2
'''
def prtlenergy(path, ind,sp):
    u = readprtl(path,'u'+sp,ind)
    v = readprtl(path,'v'+sp,ind)
    w = readprtl(path,'w'+sp,ind)
    # Now calculate LF of the particles in downstream restframe
    # gamma = sqrt((beta*gamma)^2+1.)
    energy = np.sqrt(u**2+v**2+w**2+1.)
    # If they are electrons this already the energy in units m_e c^2.
    # Otherwise...
    if sp=='i':
        energy *= readparam(path,'mi')/readparam(path,'me')
    return energy

In [16]:
'''
Calculate electron/ion temperature (work for no relativistic case)
in unit k T_s/m_e c^2
'''
def prtl_tmp(path, ind,sp,xbins,direct='x',xL=0,xR=1000,shock=1):
    # read 4-vel from prtl file
    u  = readprtl(path,'u'+sp,ind)
    v  = readprtl(path,'v'+sp,ind)
    w  = readprtl(path,'w'+sp,ind)
    ch = readprtl(path,'ch'+sp,ind)
    gamprt=np.sqrt(1.+u**2+v**2+w**2)
    # 3-velocity(/c)
    u  = u/gamprt*ch
    v  = v/gamprt*ch
    w  = w/gamprt*ch
    xp = readprtl(path,'x'+sp,ind)/readparam(path,'c_omp')
    if shock==1:
        ishf,xshf,sh_speed = shfind(path,ind)
        xL += xshf
        xR += xshf
    x_values = np.linspace(xL,xR,xbins+1)
    
    btheta   = readparam(path,'btheta')
    Tmp = np.linspace(0,0,xbins)
    for i in range(xbins):
        ma   =  (xp >= x_values[i]) & (xp < x_values[i+1])
        if direct=='x':
            #Tmp[i] = np.mean((u[ma]-np.mean(u[ma]))**2)
            Tmp[i] = np.mean(u[ma]**2) - np.mean(u[ma])**2
        elif direct=='y':
            #Tmp[i] = np.mean((v[ma]-np.mean(v[ma]))**2)
            Tmp[i] = np.mean(v[ma]**2)-np.mean(v[ma])**2
        elif direct =='z':
            #Tmp[i] = np.mean((w[ma]-np.mean(w[ma]))**2)
            Tmp[i] == np.mean(w[ma]**2)-np.mean(w[ma])**2
        elif direct=='prp':
            vprp = -u[ma]*np.sin(btheta)+v[ma]*np.cos(btheta)
            Tmp[i] = (np.mean(vprp**2)-np.mean(vprp)**2+ \
                     np.mean(w[ma]**2)-np.mean(w[ma])**2)/2.0
        elif direct=='prl':
            vprl =  u[ma]*np.cos(btheta)+v[ma]*np.sin(btheta)
            Tmp[i] = np.mean(vprl**2) - np.mean(vprl)**2
        elif direct=='tot':
            #vtot = np.sqrt(u[ma]**2+v[ma]**2+w[ma]**2)
            #Tmp[i] = np.mean(vtot**2)-np.mean(vtot)**2
            Tmp[i] = (np.mean(u[ma]**2+v[ma]**2+w[ma]**2) - np.mean(u[ma])**2
                  - np.mean(v[ma])**2-np.mean(w[ma])**2)/3.
        else:
            print ('choose right component: x,y,z,prl,prp,tot!\n')
            
        
    if sp=='i':
        return Tmp*readparam(path,'mi')/readparam(path,'me')
    else:
        return Tmp

In [17]:
'''
Calculate electron/ion temperature (work for maxwell jutner distribution case)
in unit k T_s/m_e c^2
'''
def prtl_tmp_rel(path,ind,sp,xbins,xL=0, xR=1000,shock=1,frame='upstream'):
    # read 4-vel from prtl file
    u  = readprtl(path,'u'+sp,ind)
    v  = readprtl(path,'v'+sp,ind)
    w  = readprtl(path,'w'+sp,ind)
    ch = readprtl(path,'ch'+sp,ind)
    xp = readprtl(path,'x'+sp,ind)/readparam(path,'c_omp')
    ge =np.sqrt(1.+u**2+v**2+w**2)
    if frame=='upstream':
        # Boost into the upstream frame:
        betaBoost = -np.sqrt(1-1./readparam(path,'gamma0')**2)
        rap_boost = np.arccosh(readparam(path,'gamma0'))#Boost rapidity
        rap_e = np.arccosh(ge)
        ge = ge*readparam(path,'gamma0')-np.sign(u)*np.sign(betaBoost)*np.sinh(rap_e)* \
            np.sinh(rap_boost)/np.sqrt(1+(v/u)**2+(w/u)**2)
    # select region
    if shock==1:
        ishf,xshf,sh_speed = shfind(path,ind)
        xL += xshf
        xR += xshf
    x_values = np.linspace(xL,xR,xbins+1)
    
    Tmp = np.linspace(0,0,xbins)
    for i in range(xbins):
        ma     =  (xp >= x_values[i]) & (xp < x_values[i+1])
        Tmp[i] = np.sqrt(np.mean((ge[ma]-np.mean(ge[ma]))**2))/3.
    if sp=='i':
        return Tmp*readparam('mi')/readparam('me')
    else:
        return Tmp

In [18]:
'''
Calculate electron/ion temperature (work for general relativistic case)
in unit k T_s/m_e c^2
'''
def prtl_tmp_general_rel(path,ind,sp, xL=0, xR=1000,shock=1,frame='rest',pres=0):
    
    xsl   = readspect(path,'xsl',ind)/readparam(path,'c_omp')
    spece = readspect(path,'spece',ind,frame)
    if sp=='i':
        spece = readspect(path,'specp',ind,frame)
 
    # Load gamma-1 spectrum
    gamma = readspect(path,'gamma',ind)
    # mometum := gamma*beta
    momentum = np.sqrt((gamma+1.)**2-1.)
    
    for i in range(len(xsl)):
        spece[:,i] *= gamma   

    # normalized energy spectra ( f = (dN/dE)/N )
    dgamma = np.empty(len(gamma))
    dmom = np.empty(len(gamma))
    delta  = np.log10(gamma[-1]/gamma[0])/len(gamma) 
    for i in range(len(dgamma)):
        dgamma[i] = gamma[i]*(10.**delta-1.)

    # select x-range from which to take the spectrum reletive to shock front
    if xL >= xR:
        xR = xL + 1.
        print ('Error in spectrum range: xL<xR!')
    # ishf=index, xshf=position
    ishf,xshf,sh_speed = shfind(path,ind)
    # case=0 xL-xR regions are relative to shock.
    if shock==1:
        eL = np.searchsorted(xsl,xL+xshf)
        eR = np.searchsorted(xsl,xR+xshf,side='right')
    else:
        eL = np.searchsorted(xsl,xL)
        eR = np.searchsorted(xsl,xR,side='right')

    # energy distribution, f(E) = (dn/dE)/N
    fe = np.empty(len(gamma))
    norme = np.ones(len(xsl))

    # total species in each linear x bin
    for i in range(len(norme)):
        norme[i] = sum(spece[:,i])
        
    for k in range(len(fe)):
        fe[k] = sum(spece[k][eL:eR])

    if sum(norme[eL:eR])>0:
        if(pres==0):
            fe *= 1.0/sum(norme[eL:eR])
        fe *= dgamma**(-1)
        femom = fe/(4.*np.pi*momentum)/(gamma+1.)
    else:
        print ('RUNTIME WARNING: spectrum.py cannot find electron in the integration region!')
        sys.exit()
        
        
    # now we have f(p) and p, we can calculate pressure/temperature in relativistic case
    # T/m c^2 = 4*pi/3 \int (p^4/(gamma*mc^2)) f(p)dp by assuming isotropic distribution.
    Tmp = 0.
    #num1 = 0
    for i in range(len(momentum)):
        if i>0 and i<len(momentum)-1:
            dp = (momentum[i+1]-momentum[i-1])/2.
        elif i==0:
            dp = momentum[i+1]-momentum[i]
        elif i== len(momentum)-1:
            dp = momentum[i]-momentum[i-1]
            
        Tmp += momentum[i]**4/(gamma[i]+1)*femom[i]*dp
        #if sp=='e':
        #    if momentum[i]>1.:
        #        Tmp += momentum[i]**4/(gamma[i]+1)*femom[i]*dp
        #else:
        #     Tmp += momentum[i]**4/(gamma[i]+1)*femom[i]*dp
        #num1 += momentum[i]**2*femom[i]*dp
    Tmp *= 4*np.pi/3.
    
    #print ('zeroth momentum:',num1*4*np.pi)
    if sp=='i':
        return Tmp*readparam('mi')/readparam('me')
    else:
        return Tmp


In [19]:
'''
calculate CR injection efficiency
'''
def inj_efficiency(path,ind,xL,xR,crit):
    gammae = readprtl(path,'gammae',ind)
    xe     = readprtl(path,'xe',ind)/readparam(path,'c_omp')
    ishf,xshf,sh_speed = shfind(path,ind)
    mask   = ((xe<xR+xshf) & (xe>xL+xshf))
    gammae_inrange = gammae[mask]
    return  np.sum(gammae_inrange>crit)/np.sum(gammae_inrange)

In [20]:
'''
calculate x-direction momentum
frame could be down/up/shock
'''
def v3d(path,ind,sp,xbins,pval,frame):
    dens = readflds(path,'dens',ind)
    if sp=='i':
        dens = readflds(path,'densi',ind)
    ue = readprtl(path,'u'+sp,ind)
    ve = readprtl(path,'v'+sp,ind)
    we = readprtl(path,'w'+sp,ind)
    ge = np.sqrt(ue**2+ve**2+we**2+1.)
    # 3-velocity(/c)
    vex = ue/ge 
    vey = ve/ge
    vez = we/ge 
    xe = readprtl(path,'x'+sp,ind)/readparam(path,'c_omp')

    betaBoost = 0
    if frame=='up':
        gamma0 = readparam(path,'gamma0')
        betaBoost = -np.sqrt(1-1./gamma0**2)
    elif frame=='shock':
        ish,xshf,sh_speed=shfind(ind) 
        betaBoost = -sh_speed
        gamma0 = 1./np.sqrt(1-betaBoost**2)
    if betaBoost !=0:
        tmp_ehelper = 1-vex*betaBoost
        vex = (vex-betaBoost)/tmp_ehelper
        vey = vey/gamma0/tmp_ehelper
        vez = vez/gamma0/tmp_ehelper

    xaxis = np.linspace(0,dens.shape[1]/readparam(path,'c_omp')*readparam(path,'istep'),xbins+1)
    v3d = np.zeros((xbins,1))
    for i in range(xbins):
        ma = ((xe >= xaxis[i]) & (xe<xaxis[i+1])) 
        if pval=='x':
            v3d[i] = np.average(vex[ma])
        if pval=='y':
            v3d[i] = np.average(vey[ma])
        if pval=='z': 
            v3d[i] = np.average(vez[ma])
    
    return xaxis[0:-1],v3d

In [21]:
'''
tables are integrated PDFs for initialization of maxwellians for electrons and ions
delgam in the input file sets the temperature in delta gamma, here it is converted 
between species by multiplying by mass.
!! gamma_table is gamma -1 table
'''
def init_maxw_table(delgam,pdf_sz):
    maxg        = (delgam)*20+1.
    gamma_table = np.linspace(0,0,pdf_sz)
    pdf_table   = np.linspace(0,0,pdf_sz)
    for i in range(pdf_sz):
        gamma_table[i]=(maxg-1.)/(pdf_sz-1)*i
    # a safer way of writing gamma*sqrt(gamma^2-1)*exp(-(gamma-1)/delgam)
    func=(gamma_table+1.)*np.sqrt(gamma_table*(gamma_table+2.))*np.exp(-(gamma_table)/delgam)  
    pdf_table[0]=0.
    for i in range(1,pdf_sz):
        pdf_table[i]=sum(func[0:i+1])
    #normalize pdf_table         
    pdf_table=pdf_table/pdf_table[pdf_sz-1]
    return gamma_table,pdf_table

In [22]:
# Maxwell distribution for the particles
def init_rel_maxwell(gamma0,pdf_sz,gamma_table,pdf_table):
    gamma0mag = np.abs(gamma0)
    beta_drift = np.sign(gamma0)*np.sqrt(1.-1./max(abs(gamma0),1.)**2)
    
    random.seed(time.time())    
    rannum = random.uniform(0,1)
    if (rannum==1.):
        rannum = random.uniform(0,1)
    gam=0. #gam = gamma-1., only kinetic energy to avoid underflows
    #choose gamma from the table
    for i in range(pdf_sz):               
        if(i==pdf_sz-1):
            gam=gamma_table[pdf_sz-1]
        if(rannum > pdf_table[i] and rannum < pdf_table[i+1]):                      
            gam=gamma_table[i]+(gamma_table[i+1]-gamma_table[i]) \
            /(pdf_table[i+1]-pdf_table[i])*(rannum-pdf_table[i])
            break
    
    pcosth = 2*random.uniform(0,1)-1
    pphi   = random.uniform(0,1)*2*np.pi
    psinth = np.sqrt(1-pcosth**2)   
    
    # v0t = beta
    v0t = np.sqrt((gam)*(gam+2.))/(1.+gam)     
    ut1 = v0t*psinth*np.cos(pphi)
    vt1 = v0t*psinth*np.sin(pphi)
    wt1 = v0t*pcosth
    
    # aka Zenitani 2015, u = gam*v, including c factor
    ptx = (1.+gam)*ut1
    pty = (1.+gam)*vt1
    ptz = (1.+gam)*wt1
    # flipping method aka Zenitani 2015, TABLE II
    X7  = random.uniform(0,1)
    if (-beta_drift*ut1>X7):
        ptx = -ptx
    # particles will have average velocity beta0 (with sign)    
    px1 = (ptx+beta_drift*(gam+1.))*gamma0mag #this will include the sign of gamma0
    py1 = pty
    pz1 = ptz
    
    gam1=np.sqrt(1+px1**2+py1**2+pz1**2)

    u=px1/gam1
    v=py1/gam1
    w=pz1/gam1
    return gam1,u,v,w

In [23]:
# delgam = k T/m c^2
def generate_maxwell_dist(num,delgam,pdf_sz):
    gamma_table,pdf_table = init_maxw_table(delgam,pdf_sz)
    ux  = np.empty(num) # beta nor gamma*beta
    uy  = np.empty(num)
    uz  = np.empty(num)
    gam0= np.empty(num)
    for i in range(num):
        gam0[i],ux[i],uy[i],uz[i] = init_rel_maxwell(1.,pdf_sz,gamma_table,pdf_table)
    return gam0, ux, uy, uz

In [24]:
'''
calculate cross field potential
'''
def cross_field_potential(dpath,ind,param,xL,xR):
    pval   = readflds(dpath,param,ind)
    pfield = pval[pval.shape[0]/2,:]/readparam(dpath,'b0',ind)
    pval   = pval[3:pval.shape[0]-2,:]
    if pval.shape[0]>10:
        pfield = np.mean(pval,axis=0)/readparam(dpath,'b0',ind)
    xval  = np.linspace(0,pval.shape[1]/readparam(dpath,'c_omp')*readparam(dpath,'istep'),pval.shape[1])
    ishf,xshf,_= shfind(dpath,ind)
    in_range = (xval>=xshf+xL) & (xval<=xshf+xR) 
    
    yfield = pfield[in_range]
    xaxis  = xval[in_range]
    
    int_potential = integrate.cumtrapz(yfield[::-1], xaxis[::-1], initial=0)
    Poten_reverse = -int_potential*cal_param(dpath,'Va')*np.sqrt(readparam(dpath,'mi')/readparam(dpath,'me'))
    Poten = Poten_reverse[::-1]
    in_range = (xaxis>=xshf+xL) & (xaxis<=xshf) 
    return np.mean(Poten[in_range])

In [25]:
'''
density increase of the shock
'''
def denratio(dpath,ind,xL,xR):
    ishf,xshf,sh_speed = shfind(dpath,ind) # shock speed doesn't work if cutting downstream/upstream
    dens  = readflds(dpath,'dens',ind)
    xL = int((xL+xshf)*readparam(dpath,'c_omp')/readparam(dpath,'istep'))
    xR = int((xR+xshf)*readparam(dpath,'c_omp')/readparam(dpath,'istep'))
    den1d = dens[dens.shape[0]/2,:]
    return np.mean(den1d[xL:ishf])/np.mean(den1d[ishf:xR])


In [26]:
'''
magnetic field increase in the shock
'''
def fldsratio(dpath,ind,xL,xR):
    ishf,xshf,_= shfind(dpath,ind) # shock speed doesn't work if cutting downstream/upstream
    dens  = readflds(dpath,'dens',ind)
    B  = np.sqrt(readflds(dpath,'bx',ind)**2 + readflds(dpath,'by',ind)**2 + readflds(dpath,'bz',ind)**2)
    xL = int((xL+xshf)*readparam(dpath,'c_omp')/readparam(dpath,'istep'))
    xR = int((xR+xshf)*readparam(dpath,'c_omp')/readparam(dpath,'istep'))
    flds1d = B[B.shape[0]/2,:]
    if B.shape[0]>10:
        flds1d = np.mean(B[3:B.shape[0]-2,:],axis=0)
    return np.mean(flds1d[xL:ishf])/np.mean(flds1d[ishf+100:xR])
